<a href="https://colab.research.google.com/github/AmanAryan007/Driving-licencce-checker/blob/main/scrapdata2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt-get install chromium chromium-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [770 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ub

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
import re
import json
from lxml import html  # Importing the html module
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from PIL import Image
from io import BytesIO

# def save_captcha_image(captcha_image_url, captcha_image_path):
#     response = requests.get("https://parivahan.gov.in" + captcha_image_url)
#     with open(captcha_image_path, 'wb') as f:
#         f.write(response.content)

# Initialize WebDriver
driver = web_driver()

driver.get("https://parivahan.gov.in/rcdlstatus/?pur_cd=101")
# Wait for the page to load
time.sleep(5)

# Take screenshot using Selenium WebDriver
screenshot = driver.get_screenshot_as_png()

# Open the screenshot image using Pillow
img = Image.open(BytesIO(screenshot))

# Get the size of the original image
img_width, img_height = img.size

# Define the amount of cropping from each side (in pixels)
crop_top = 350
crop_bottom = 800
crop_left = 960
crop_right = 830

# Calculate the new dimensions after cropping
new_width = img_width - crop_left - crop_right
new_height = img_height - crop_top - crop_bottom

# Define the crop box (left, top, right, bottom)
crop_box = (crop_left, crop_top, img_width - crop_right, img_height - crop_bottom)

try:
    # Crop the image using the defined box
    cropped_img = img.crop(crop_box)

    # Resize the cropped image to the specified dimensions
    resized_img = cropped_img.resize((new_width, new_height))

    # Save the resized image
    resized_img.save("resized_image.png")

    # Show the resized image (optional)
    resized_img.show()
except Exception as e:
    print("Error:", e)


dl_number = "MH0320140015542"
dob = "21-06-1992"

# Find and fill DL Number field
dl_number_field = driver.find_element(By.NAME, "form_rcdl:tf_dlNO")
dl_number_field.clear()
dl_number_field.send_keys(dl_number)
# dob_field.send_keys(dob)  # Fill in the new Date of Birth
dob_field = driver.find_element(By.ID, "form_rcdl:tf_dob_input")
driver.execute_script("arguments[0].setAttribute('value', arguments[1])", dob_field, dob)


# Manually enter CAPTCHA
captcha_input = input("Enter the CAPTCHA from captcha_image.png: ")

# Fill CAPTCHA field
captcha_field = driver.find_element(By.NAME, "form_rcdl:j_idt31:CaptchaID")
captcha_field.send_keys(captcha_input)

# Submit the form
check_status_button = driver.find_element(By.ID, "form_rcdl:j_idt41")
check_status_button.click()

# Wait for the results to load
time.sleep(5)

# data = {}# Find the div element by ID
detail_div = driver.find_element(By.ID, "form_rcdl:j_idt63")

# Extract text content from the div
details = detail_div.text.strip()

# Extracting necessary information using regular expressions
dl_number = re.search(r'Details Of Driving License: (.+)', details).group(1)
current_status = re.search(r'Current Status (.+)', details).group(1)
holder_name = re.search(r"Holder's Name (.+)", details).group(1)
old_new_dl_no = re.search(r'Old \/ New DL No\. (.+)', details).group(1)
source_of_data = re.search(r'Source Of Data (.+)', details).group(1)
initial_issue_date = re.search(r'Initial Issue Date (.+)', details).group(1)
initial_issuing_office = re.search(r'Initial Issuing Office (.+)', details).group(1)
last_endorsed_date = re.search(r'Last Endorsed Date (.+)', details).group(1)
last_endorsed_office = re.search(r'Last Endorsed Office (.+)', details).group(1)
last_completed_transaction = re.search(r'Last Completed Transaction (.+)', details).group(1)
non_transport_validity = re.search(r'Non-Transport From: (.+) To: (.+)', details)
transport_validity = re.search(r'Transport From: (.+) To: (.+)', details)
hazardous_valid_till = re.search(r'Hazardous Valid Till (.+)', details).group(1)
hill_valid_till = re.search(r'Hill Valid Till (.+)', details).group(1)

# Extracting Class Of Vehicle Details
class_of_vehicle_details = re.findall(r'(\w+) (\w+-?\w*) (\d{2}-\w{3}-\d{4})', details)

# Building the JSON object
data = {
    "current_status": current_status.strip(),
    "holder_name": holder_name.strip(),
    "old_new_dl_no": old_new_dl_no.strip(),
    "source_of_data": source_of_data.strip(),
    "initial_issue_date": initial_issue_date.strip(),
    "initial_issuing_office": initial_issuing_office.strip(),
    "last_endorsed_date": last_endorsed_date.strip(),
    "last_endorsed_office": last_endorsed_office.strip(),
    "last_completed_transaction": last_completed_transaction.strip(),
    "driving_license_validity_details": {
        "non_transport": {
            "valid_from": non_transport_validity.group(1).strip(),
            "valid_upto": non_transport_validity.group(2).strip()
        },
        "transport": {
            "valid_from": transport_validity.group(1).strip(),
            "valid_upto": transport_validity.group(2).strip()
        }
    },
    "hazardous_valid_till": hazardous_valid_till.strip(),
    "hill_valid_till": hill_valid_till.strip(),
    "class_of_vehicle_details": [
        {
            "cov_category": item[0].strip(),
            "vehicle_category": item[1].strip(),
            "cov_issue_date": item[2].strip()
        } for item in class_of_vehicle_details
    ]
}

# Convert dictionary to JSON format
json_data = json.dumps(data, indent=2)

# Print the JSON data
print(json_data)


driver.quit()



Enter the CAPTCHA from captcha_image.png: xwe6d
{
  "current_status": "ACTIVE",
  "holder_name": "A*M*DA*I S*A*K*",
  "old_new_dl_no": "NA",
  "source_of_data": "SARATHI",
  "initial_issue_date": "25-Apr-2014",
  "initial_issuing_office": "RTO,MUMBAI (EAST)",
  "last_endorsed_date": "06-Aug-2018",
  "last_endorsed_office": "RTO,MUMBAI (EAST)",
  "last_completed_transaction": "RENEWAL OF DL , ISSUE OF DRIVING LICENCE",
  "driving_license_validity_details": {
    "non_transport": {
      "valid_from": "25-Apr-2014",
      "valid_upto": "24-Apr-2034"
    },
    "transport": {
      "valid_from": "25-Apr-2014",
      "valid_upto": "24-Apr-2034"
    }
  },
  "hazardous_valid_till": "NA Hill Valid Till NA",
  "hill_valid_till": "NA",
  "class_of_vehicle_details": [
    {
      "cov_category": "Issue",
      "vehicle_category": "Date",
      "cov_issue_date": "25-Apr-2014"
    },
    {
      "cov_category": "Endorsed",
      "vehicle_category": "Date",
      "cov_issue_date": "06-Aug-2018"
  